# Publication queries 

This notebook contains a collection of common publication queries for [Dimensions on Google BigQuery](https://docs.dimensions.ai/bigquery/).

For more background, see also the [publications data model](https://docs.dimensions.ai/bigquery/datasource-publications.html). 

## Prerequisites

This notebook assumes that you have [verifed your connection](https://digital-science.github.io/dimensions-gbq-lab/cookbooks/1-Verifying-your-connection.html) to Dimensions on Google BigQuery and have basic familiarity with the [Google Cloud Platform](https://docs.dimensions.ai/bigquery/gcp-setup.html) concepts. 

The following code will load the Python BigQuery library and authenticate you as a valid user.  

In [34]:
!pip install google-cloud-bigquery -U --quiet
%load_ext google.cloud.bigquery

import sys
print("==\nAuthenticating...")
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    print('..done (method: Colab)')
else:
    from google.cloud import bigquery
    print('..done (method: local credentials)')
    
#
# PLEASE UPDATE USING YOUR CLOUD PROJECT ID (= the 'billing' account)
#

MY_PROJECT_ID = "ds-data-solutions-gbq"

print("==\nTesting connection..")
client = bigquery.Client(project=MY_PROJECT_ID)
test = client.query("""
    SELECT COUNT(*) as pubs 
    from `dimensions-ai.data_analytics.publications`
    """)
rows = [x for x in test.result()]
print("...success!")
print("Total publications in Dimensions: ", rows[0]['pubs'])

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
==
Authenticating...
..done (method: local credentials)
==
Testing connection..
...success!
Total publications in Dimensions:  115963650


## 1. Top publications by Altmetric score and research organization 



In [9]:
%%bigquery --project $MY_PROJECT_ID

-- Top 5 pubs by Altmetric Score for GRID ID grid.4991.5 in the year 2020

SELECT
  id,
  title.preferred as title,
  ARRAY_LENGTH(authors) as authors,
  altmetrics.score as altmetrics_score
FROM
  `dimensions-ai.data_analytics.publications`
WHERE
  year = 2020 AND 'grid.4991.5' in UNNEST(research_orgs)
ORDER BY
  altmetrics.score desc
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:02<00:00,  1.99rows/s]


,id,title,authors,altmetrics_score
0,pub.1130340155,Two metres or one: what is the evidence for ph...,6,15626
1,pub.1129493369,Safety and immunogenicity of the ChAdOx1 nCoV-...,366,15382
2,pub.1127239818,Remdesivir in adults with severe COVID-19: a r...,46,12139
3,pub.1133359801,Safety and efficacy of the ChAdOx1 nCoV-19 vac...,766,11111
4,pub.1131721397,Scientific consensus on the COVID-19 pandemic:...,31,10429


## 1. Working with Publications dates 

Each publication has various dates available. 

* `date`, `year`, `date_normal`, `date_online`, `date_print` refer to the publication object. See the [documentation](https://docs.dimensions.ai/bigquery/datasource-publications.html) to find out more about their meaning. 
* `date_imported_gbq` refers to when this record was last added to GBQ - this date can be handy if you want to synchronize an external data source to GBQ. 
* `date_inserted`: this refers to when this records was originally added to Dimensions (if the records gets adjusted later, it doesn't change). 

### Comparing date fields

In [13]:
%%bigquery --project $MY_PROJECT_ID

SELECT doi,
       date,
       date_normal,
       year,
       date_online,
       date_print,
       date_imported_gbq,
       date_inserted
FROM   `dimensions-ai.data_analytics.publications`
WHERE  year = 2010
       AND journal.id = "jour.1115214"
ORDER  BY citations_count DESC
LIMIT  10 

Downloading: 100%|██████████| 10/10 [00:02<00:00,  3.86rows/s]


,doi,date,date_normal,year,date_online,date_print,date_imported_gbq,date_inserted
0,10.1038/nbt.1621,2010-05-02,2010-05-02,2010,2010-05-02,2010-05,2021-02-10 01:09:29+00:00,2017-08-31 12:50:56+00:00
1,10.1038/nbt.1630,2010-05-02,2010-05-02,2010,2010-05-02,2010-05,2021-02-10 01:09:29+00:00,2017-08-31 12:50:56+00:00
2,10.1038/nbt.1614,2010-03,2010-03-01,2010,None,2010-03,2021-02-10 01:09:29+00:00,2017-08-31 12:50:56+00:00
3,10.1038/nbt.1685,2010-10-13,2010-10-13,2010,2010-10-13,2010-10,2021-02-10 00:53:56+00:00,2017-08-31 12:50:56+00:00
4,10.1038/nbt1210-1248,2010-12-07,2010-12-07,2010,2010-12-07,2010-12,2021-02-10 00:53:56+00:00,2017-08-31 12:50:56+00:00
5,10.1038/nbt.1755,2010-12-22,2010-12-22,2010,2010-12-22,2011-02,2021-02-10 01:09:29+00:00,2017-08-31 12:50:56+00:00
6,10.1038/nbt1010-1045,2010-10-13,2010-10-13,2010,2010-10-13,2010-10,2021-02-10 00:53:56+00:00,2017-08-31 12:50:56+00:00
7,10.1038/nbt.1633,2010-05-02,2010-05-02,2010,2010-05-02,2010-05,2021-02-10 00:53:56+00:00,2017-08-31 12:50:56+00:00
8,10.1038/nbt.1667,2010-07-19,2010-07-19,2010,2010-07-19,2010-08,2021-02-10 01:09:29+00:00,2017-08-31 12:50:56+00:00
9,10.1038/nbt.1641,2010-05-23,2010-05-23,2010,2010-05-23,2010-06,2021-02-10 00:53:56+00:00,2017-08-31 12:50:56+00:00


### Number of publications added to Dimensions by month

In [14]:
%%bigquery --project $MY_PROJECT_ID

SELECT 
  DATETIME_TRUNC(DATETIME(date_inserted), MONTH) as date,
  COUNT(id) as countDim
FROM
  `dimensions-ai.data_analytics.publications`
GROUP BY date  
ORDER BY date DESC
LIMIT 5




Downloading: 100%|██████████| 5/5 [00:02<00:00,  2.13rows/s]


,date,countDim
0,2021-02-01,174570
1,2021-01-01,685667
2,2020-12-01,820007
3,2020-11-01,573519
4,2020-10-01,718132


## 2. Working with NESTED fields 

UNNEST are implicit 'cross-join' queries, hence only records that have some value in the nested column are represented

For example, the query below return less publications that then ones available, because only the ones with `research_org_country_names` are included (= cross join)

In [45]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT p.id) AS tot_articles
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(research_org_country_names) AS research_org_country_names
WHERE
  year = 2000

Downloading: 100%|██████████| 1/1 [00:04<00:00,  4.60s/rows]


,tot_articles
0,1060342


As a test, we can run the query without the UNNEST clause

In [44]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT p.id) AS tot_articles
FROM
  `dimensions-ai.data_analytics.publications` p
WHERE
  year = 2000

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.97s/rows]


,tot_articles
0,1759389


So how can we get all the records out? 

If you want to get all records, then **LEFT JOIN is the way to go** in this case

In [15]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT p.id) AS tot_articles
FROM
  `dimensions-ai.data_analytics.publications` p
LEFT JOIN
  UNNEST(research_org_country_names) AS research_org_country_names
WHERE
  year = 2000

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.56s/rows]


,tot_articles
0,1759389


## 3. Generate a list of publication authors by flattening/concatenating nested data

IE Flattening an array of objects into a string

In [16]:
%%bigquery --project $MY_PROJECT_ID

SELECT p.id,
       ARRAY_TO_STRING(
       (
              SELECT ARRAY
                     (
                            select CONCAT(first_name, " ", last_name)
                            from   UNNEST(p.authors)) ), '; ') AS authors_list
FROM   `dimensions-ai.data_analytics.publications` p
WHERE  p.id = 'pub.1132070778'

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.53s/rows]


,id,authors_list
0,pub.1132070778,O Grånäs; A Mocellin; E S Cardoso; F Burmeiste...


## 4. Generate a list of publication categories by flattening/concatenating nested data


In [17]:
%%bigquery --project $MY_PROJECT_ID

SELECT p.id,
       ARRAY_TO_STRING(
       (
              SELECT ARRAY
                     (
                            SELECT name
                            FROM   UNNEST(p.category_for.first_level.FULL)) ), '; ') AS categories_list
FROM   `dimensions-ai.data_analytics.publications` p
WHERE  p.id = 'pub.1132070778'

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.55s/rows]


,id,categories_list
0,pub.1132070778,Physical Sciences; Chemical Sciences


## 5. Number of publications per SDG category

In [19]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT p.id) AS tot,
  sdg.name
FROM `dimensions-ai.data_analytics.publications` p,
  UNNEST(category_sdg.full) sdg
GROUP BY sdg.name
LIMIT
  5

Downloading: 100%|██████████| 5/5 [00:02<00:00,  2.11rows/s]


,tot,name
0,184789,Reduced Inequalities
1,610656,Quality Education
2,137256,Zero Hunger
3,24975,Gender Equality
4,11830,Partnerships for the Goals


## 6. Publications count per FoR category, total and percentage against total

In [18]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  cat.name,
  COUNT(DISTINCT p.id) AS pubs_global,
  ROUND ((COUNT(DISTINCT p.id) * 100 /(
      SELECT
        COUNT(*)
      FROM
        `dimensions-ai.data_analytics.publications`)), 2 ) AS pubs_global_pc
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(category_for.first_level.full) cat
GROUP BY
  cat.name

Downloading: 100%|██████████| 22/22 [00:02<00:00,  9.38rows/s]


,name,pubs_global,pubs_global_pc
0,"Language, Communication and Culture",2494744,2.15
1,Earth Sciences,2027739,1.75
2,Agricultural and Veterinary Sciences,2085752,1.80
3,Philosophy and Religious Studies,1662674,1.43
4,Medical and Health Sciences,29853801,25.74
5,Economics,1722795,1.49
6,"Commerce, Management, Tourism and Services",1792537,1.55
7,History and Archaeology,2333998,2.01
8,Technology,1932511,1.67
9,Biological Sciences,8922205,7.69


## 7. Finding Journals using string matching

In [20]:
%%bigquery --project $MY_PROJECT_ID

SELECT COUNT(*) AS pubs,
  journal.id,
  journal.title,
  journal.issn,
  journal.eissn,
  publisher.name
FROM
  `dimensions-ai.data_analytics.publications`
WHERE
  LOWER( journal.title ) LIKE CONCAT('%medicine%')
GROUP BY 2, 3, 4, 5, 6
ORDER BY pubs DESC
LIMIT 20

Downloading: 100%|██████████| 20/20 [00:02<00:00,  8.17rows/s]


,pubs,id,title,issn,eissn,name
0,168620,jour.1014075,New England Journal of Medicine,0028-4793,1533-4406,Massachusetts Medical Society
1,83860,jour.1011551,Medicine & Science in Sports & Exercise,0195-9131,1530-0315,Wolters Kluwer
2,58617,jour.1017222,Annals of Internal Medicine,0003-4819,1539-3704,American College of Physicians
3,52792,jour.1312267,Journal of the Royal Society of Medicine,0141-0768,1758-1095,SAGE Publications
4,52248,jour.1017256,JAMA Internal Medicine,2168-6106,2168-6114,American Medical Association (AMA)
5,47104,jour.1027092,Experimental Biology and Medicine,1535-3702,1535-3699,SAGE Publications
6,46274,jour.1016342,Critical Care Medicine,0090-3493,1530-0293,Wolters Kluwer
7,37632,jour.1057918,Journal of Molecular Medicine,0946-2716,1432-1440,Springer Nature
8,34891,jour.1017275,Arizona Medicine,0093-0415,1476-2978,None
9,31068,jour.1014535,The American Journal of Medicine,0002-9343,1555-7162,Elsevier


## 8. Finding articles matching a specific affiliation string

In [21]:
%%bigquery --project $MY_PROJECT_ID

SELECT id,
       aff.grid_id,
       aff.raw_affiliation
FROM   `dimensions-ai.data_analytics.publications`,
       UNNEST(authors) auth,
       UNNEST(auth.affiliations_address) AS aff
WHERE  year = 2020
AND    aff.grid_id = "grid.69566.3a"
AND    LOWER(aff.raw_affiliation) LIKE "%school of medicine%"

Downloading: 100%|██████████| 5920/5920 [00:02<00:00, 2233.04rows/s]


,id,grid_id,raw_affiliation
0,pub.1112041600,grid.69566.3a,"5Department of Neurosurgery, Tohoku University..."
1,pub.1112041600,grid.69566.3a,"6Division of Epidemiology, Department of Healt..."
2,pub.1117164397,grid.69566.3a,"Division of Cardiovascular Surgery, Tohoku Uni..."
3,pub.1119863526,grid.69566.3a,Division of Emergency and Critical Care Medici...
4,pub.1122526898,grid.69566.3a,"Department of Neurological Science, Tohoku Uni..."
...,...,...,...
5915,pub.1128974183,grid.69566.3a,"Department of Organ Anatomy, Tohoku University..."
5916,pub.1128974183,grid.69566.3a,"Department of Organ Anatomy, Tohoku University..."
5917,pub.1128974183,grid.69566.3a,"Department of Organ Anatomy, Tohoku University..."
5918,pub.1128977047,grid.69566.3a,"Department of Molecular Pathology, Tohoku Univ..."


### 8.1 Variant: get unique publication records with affiliation count 

In [22]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(aff) AS matching_affiliations,
  id,
  title.preferred AS title
FROM
  `dimensions-ai.data_analytics.publications`,
  UNNEST(authors) auth,
  UNNEST(auth.affiliations_address) AS aff
WHERE
  year = 2020
  AND aff.grid_id = "grid.69566.3a"
  AND LOWER(aff.raw_affiliation) LIKE "%school of medicine%"
GROUP BY
  id,
  title

Downloading: 100%|██████████| 1492/1492 [00:02<00:00, 531.33rows/s]


,matching_affiliations,id,title
0,9,pub.1124154670,P822 Genetic analysis of ulcerative colitis in...
1,8,pub.1124073702,Exploring the Novel Susceptibility Gene Varian...
2,1,pub.1124562761,Prediction of the Probability of Malignancy in...
3,2,pub.1124468935,Usefulness of a Kampo Medicine on Stress-Induc...
4,1,pub.1124922689,Qualitative investigation of the factors that ...
...,...,...,...
1487,5,pub.1131095571,Hypoketotic hypoglycemia in citrin deficiency:...
1488,2,pub.1133151054,Electronic phenotyping of heart failure from a...
1489,1,pub.1132062513,Retrospective details of false-positive and fa...
1490,12,pub.1132032426,Identification of the Novel Variants in Patien...


## 9. Select publications matching selected concepts

In [23]:
%%bigquery --project $MY_PROJECT_ID

WITH tropical_diseases AS
(
       SELECT *
       FROM   `dimensions-ai.data_analytics.publications` )
SELECT   publisher.NAME AS publisher,
         year,
         count(*) AS num_pub
FROM     tropical_diseases,
         UNNEST(tropical_diseases.concepts) c
WHERE    (
                  LOWER(c.concept) IN UNNEST(["buruli ulcer", "mycobacterium", "mycolactone", "bairnsdale ulcer"])
         OR       REGEXP_CONTAINS(title.preferred, r"(?i)/buruli ulcer|mycobacterium|mycolactone|bairnsdale ulcer/"))
AND      year >= 2010
AND      publisher IS NOT NULL
GROUP BY publisher, year
ORDER BY num_pub DESC,
         year,
         publisher LIMIT 10

Downloading: 100%|██████████| 10/10 [00:02<00:00,  3.98rows/s]


,publisher,year,num_pub
0,Elsevier,2020,31812
1,Elsevier,2018,29580
2,Elsevier,2019,28941
3,Elsevier,2017,28415
4,Elsevier,2015,27301
5,Elsevier,2011,25758
6,Elsevier,2016,25149
7,Elsevier,2013,23209
8,Elsevier,2014,23100
9,Springer Nature,2019,22072


## 10. Count of corresponding authors by publisher 

In [24]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT id) AS tot,
  publisher.name
FROM
  `dimensions-ai.data_analytics.publications`,
  UNNEST(authors) aff
WHERE
  aff.corresponding IS TRUE
  AND publisher.name IS NOT NULL
GROUP BY
  publisher.name
ORDER BY
  tot DESC

Downloading: 100%|██████████| 421/421 [00:02<00:00, 208.14rows/s]


,tot,name
0,8733776,Elsevier
1,5885408,Springer Nature
2,813007,Institute of Electrical and Electronics Engine...
3,683093,SAGE Publications
4,380636,MDPI
...,...,...
416,1,New York Entomological Society
417,1,Kansas Academy of Science
418,1,Gorgias Press LLC
419,1,Journal of the North Atlantic


## 11. Counting new vs recurring authors, for a specific journal

In [25]:
%%bigquery --project $MY_PROJECT_ID


WITH
  authoryear AS (
  SELECT pubs.year, author.researcher_id, COUNT(pubs.id) AS numpubs
  FROM
    `dimensions-ai.data_analytics.publications` AS pubs
  CROSS JOIN
    UNNEST(pubs.authors) AS author
  WHERE
    author.researcher_id IS NOT NULL
    AND journal.id= "jour.1115214"
  GROUP BY
    author.researcher_id, pubs.year ),

authorfirst AS (
  SELECT researcher_id, MIN(year) AS minyear
  FROM
    authoryear
  GROUP BY
    researcher_id ),

authorsummary AS (
  SELECT ay.*,
  IF
    (ay.year=af.minyear,
      TRUE,
      FALSE) AS firstyear
  FROM
    authoryear ay
  JOIN
    authorfirst af
  ON
    af.researcher_id=ay.researcher_id
  ORDER BY
    ay.researcher_id, year ),

numauthors AS (
  SELECT year, firstyear, COUNT(DISTINCT researcher_id) AS numresearchers
  FROM
    authorsummary
  WHERE
    year>2010
  GROUP BY year, firstyear )

SELECT
  year,
  SUM(CASE
      WHEN firstyear THEN numresearchers
    ELSE
    0
  END
    ) AS num_first,
  SUM(CASE
      WHEN NOT firstyear THEN numresearchers
    ELSE
    0
  END
    ) AS num_recurring
FROM numauthors
GROUP BY year
ORDER BY year


Downloading: 100%|██████████| 10/10 [00:02<00:00,  4.29rows/s]


,year,num_first,num_recurring
0,2011,1040,352
1,2012,858,373
2,2013,926,345
3,2014,1088,338
4,2015,1044,392
5,2016,1313,350
6,2017,1072,404
7,2018,1104,419
8,2019,1184,442
9,2020,1579,568


## 12. Funding by journal

In [27]:
%%bigquery --project $MY_PROJECT_ID

WITH funding AS
(
         SELECT   funding.grid_id         AS funders,
                  COUNT(id)               AS pubs,
                  COUNT(funding.grant_id) AS grants
         FROM     `dimensions-ai.data_analytics.publications`,
                  UNNEST(funding_details) AS funding
         WHERE    journal.id = "jour.1113716" -- nature medicine
         GROUP BY funders)

SELECT   funding.*,
         grid.NAME
FROM     funding
JOIN     `dimensions-ai.data_analytics.grid` grid
ON       funding.funders = grid.id
ORDER BY pubs DESC,
         grants DESC
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:02<00:00,  4.11rows/s]


,funders,pubs,grants,NAME
0,grid.48336.3a,2699,2484,National Cancer Institute
1,grid.419681.3,2008,1878,National Institute of Allergy and Infectious D...
2,grid.419635.c,1620,1564,National Institute of Diabetes and Digestive a...
3,grid.279885.9,1612,1525,National Heart Lung and Blood Institute
4,grid.416870.c,712,668,National Institute of Neurological Disorders a...
5,grid.419475.a,569,537,National Institute on Aging
6,grid.54432.34,510,447,Japan Society for the Promotion of Science
7,grid.280785.0,460,441,National Institute of General Medical Sciences
8,grid.14105.31,452,344,Medical Research Council
9,grid.270680.b,400,183,European Commission


## 13. Citations queries

### 13.1 Top N publications by citations percentile

In [28]:
%%bigquery --project $MY_PROJECT_ID

WITH pubs AS (
  SELECT
    p.id as id, 
    p.title.preferred as title,
    p.citations_count as citations,
  FROM
    `dimensions-ai.data_analytics.publications` p
  WHERE year = 2020 AND "09" IN UNNEST(category_for.first_level.codes)
),
ranked_pubs AS (
  SELECT
    p.*,
    PERCENT_RANK() OVER (ORDER BY p.citations DESC) citation_percentile
  FROM
    pubs p
)
SELECT * FROM ranked_pubs
WHERE citation_percentile <= 0.01
ORDER BY citation_percentile asc

Downloading: 100%|██████████| 7034/7034 [00:02<00:00, 2555.04rows/s]


,id,title,citations,citation_percentile
0,pub.1129408972,Estimation of total flavonoid content in propo...,881,0.000000
1,pub.1122861707,"Mercury 4.0: from visualization to analysis, d...",393,0.000001
2,pub.1125814051,Analysis and forecast of COVID-19 spreading in...,286,0.000003
3,pub.1126110231,Covid-19: automatic detection from X-ray image...,255,0.000004
4,pub.1125821215,The Role of Telehealth in Reducing the Mental ...,234,0.000006
...,...,...,...,...
7029,pub.1127509406,Hydrothermal carbonization of sewage digestate...,14,0.008954
7030,pub.1127507337,Flow and heat transfer of hybrid nanofluid ove...,14,0.008954
7031,pub.1127511048,Reinforcement learning for building controls: ...,14,0.008954
7032,pub.1126596495,Integrated Multi-satellite Retrievals for the ...,14,0.008954


### 13.2 Citations by journal, for a specific publisher 

In [29]:
%%bigquery --project $MY_PROJECT_ID

WITH publisher_pubs AS (
  SELECT id FROM `dimensions-ai.data_analytics.publications`
  WHERE publisher.id = "pblshr.1000340" AND type = "article"
)

SELECT 
  COUNT(p.id) as tot,
  p.journal.title as journal
FROM `dimensions-ai.data_analytics.publications` p, UNNEST(p.reference_ids) r
WHERE 
  p.year = 2020 AND p.type = "article"      -- restrict to articles with a published year of 2020
  AND p.publisher.id <> "pblshr.1000340"    -- where the publisher is not the same as the pusblisher above
  AND r IN (SELECT * FROM publisher_pubs)   -- the publication must reference a publishers publication
GROUP BY journal
ORDER BY tot DESC
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:02<00:00,  3.92rows/s]


,tot,journal
0,26147,Scientific Reports
1,18794,International Journal of Molecular Sciences
2,8647,Frontiers in Microbiology
3,8620,None
4,7695,Frontiers in Immunology
5,6960,International Journal of Environmental Researc...
6,6421,Nature Communications
7,6145,Cells
8,5687,Cancers
9,5006,Microorganisms


### 13.3 One-degree citation network for a single publication

In [30]:
%%bigquery --project $MY_PROJECT_ID

WITH level1 AS (
  select "pub.1099396382" as citation_from, citations.id as citation_to, 1 as level, citations.year as citation_year
  from `dimensions-ai.data_analytics.publications` p, unnest(citations) as citations
  where p.id="pub.1099396382"
),

level2 AS (
  select l.citation_to as citation_from, citations.id as citation_to, 2 as level, citations.year as citation_year
  from `dimensions-ai.data_analytics.publications` p, unnest(citations) as citations, level1 l
  where p.id = l.citation_to
)

SELECT * from level1 
UNION ALL
SELECT * from level2 

Downloading: 100%|██████████| 187/187 [00:02<00:00, 79.75rows/s]


,citation_from,citation_to,level,citation_year
0,pub.1114028205,pub.1131160226,2,2020
1,pub.1023754996,pub.1111137794,2,2019
2,pub.1023754996,pub.1119901753,2,2019
3,pub.1023754996,pub.1020574513,2,2010
4,pub.1023754996,pub.1022815437,2,2010
...,...,...,...,...
182,pub.1043374025,pub.1110816413,2,2019
183,pub.1043374025,pub.1090432296,2,2017
184,pub.1043374025,pub.1112307407,2,2019
185,pub.1043374025,pub.1028868656,2,2006


### 13.4 Incoming citations for a journal

In [31]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT id) AS totcount,  year, type
FROM
  `dimensions-ai.data_analytics.publications`
WHERE
  id IN (
  SELECT citing_pubs.id
  FROM
    `dimensions-ai.data_analytics.publications`,
    UNNEST(citations) AS citing_pubs
  WHERE journal.id = "jour.1115214" )  -- Nature Biotechnology
GROUP BY year, type
ORDER BY year, type

Downloading: 100%|██████████| 201/201 [00:02<00:00, 82.07rows/s]


,totcount,year,type
0,9,NaN,article
1,1,1924.0,article
2,1,1942.0,article
3,1,1963.0,article
4,1,1964.0,article
...,...,...,...
196,1,2021.0,book
197,612,2021.0,chapter
198,26,2021.0,monograph
199,894,2021.0,preprint


### 13.5 Outgoing citations to a journal

In [32]:
%%bigquery --project $MY_PROJECT_ID

SELECT
  COUNT(DISTINCT id) AS totcount,  year, type
FROM
  `dimensions-ai.data_analytics.publications`
WHERE
  id IN (
  SELECT
    DISTINCT reference_pubs
  FROM
    `dimensions-ai.data_analytics.publications`,
    UNNEST(reference_ids) AS reference_pubs
  WHERE
    journal.id = "jour.1115214" ) -- Nature Biotechnology
GROUP BY year, type
ORDER BY year, type

Downloading: 100%|██████████| 356/356 [00:02<00:00, 141.93rows/s]


,totcount,year,type
0,1,NaN,article
1,1,1825.0,article
2,1,1828.0,article
3,1,1853.0,article
4,1,1855.0,monograph
...,...,...,...
351,3,2019.0,proceeding
352,409,2020.0,article
353,5,2020.0,chapter
354,34,2020.0,preprint


## 14. Extracting complex publications records 

The query below combines various techniques presented in this notebook in order to extract full publication records that include both single-value metadata and unpacked lists. 

We use LEFT JOIN in order to ensure we obtain all records, not just the ones that have some value in the nested objects. 

In [37]:
%%bigquery --project $MY_PROJECT_ID

SELECT
 p.id,
 p.title.preferred AS title,
 p.doi,
 p.year,
 COALESCE(p.journal.title, p.proceedings_title.preferred, p.book_title.preferred, p.book_series_title.preferred) AS venue,
 p.type,
 p.date AS date_publication,
 p.date_inserted,
 p.altmetrics.score AS altmetrics_score,
 p.metrics.times_cited,
 grid.id AS gridid,
 grid.name AS gridname,
 grid.address.country AS gridcountry,
 grid.address.city AS gridcity,
 open_access_categories,
 cat_for.name AS category_for,
FROM
 `dimensions-ai.data_analytics.publications` p
LEFT JOIN
  UNNEST(research_orgs) AS research_orgs_grids
LEFT JOIN
 `dimensions-ai.data_analytics.grid` grid
ON
 grid.id=research_orgs_grids
LEFT JOIN
 UNNEST(p.open_access_categories) AS open_access_categories
LEFT JOIN
 UNNEST(p.category_for.first_level.full) AS cat_for
WHERE
 EXTRACT(YEAR
 FROM
   date_inserted) >= 2020

LIMIT 100

Downloading: 100%|██████████| 100/100 [00:02<00:00, 41.14rows/s]


,id,title,doi,year,venue,type,date_publication,date_inserted,altmetrics_score,times_cited,gridid,gridname,gridcountry,gridcity,open_access_categories,category_for
0,pub.1124854815,Chanoyu sandenshū,10.5479/sil.893207.39088019038405,1691,None,monograph,1691,2020-02-15 01:10:52+00:00,NaN,0,None,None,None,None,closed,None
1,pub.1124853520,Mag[istr]i Arn[al]di Devillan[ov]a Liber dictu...,10.5479/sil.169309.39088003312089,1666,None,monograph,1666,2020-02-15 01:10:52+00:00,NaN,0,None,None,None,None,oa_all,None
2,pub.1124853520,Mag[istr]i Arn[al]di Devillan[ov]a Liber dictu...,10.5479/sil.169309.39088003312089,1666,None,monograph,1666,2020-02-15 01:10:52+00:00,NaN,0,None,None,None,None,gold_bronze,None
3,pub.1127222143,A New Method of a Common-Place-Book,10.1093/oseo/instance.00263866,1706,The Clarendon Edition of the Works of John Loc...,chapter,1706,2020-04-30 18:45:39+00:00,NaN,2,None,None,None,None,closed,None
4,pub.1127222159,A Letter from the First Earl of Shaftesbury to...,10.1093/oseo/instance.00263882,1706,The Clarendon Edition of the Works of John Loc...,chapter,1706,2020-04-30 18:45:39+00:00,NaN,0,None,None,None,None,closed,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,pub.1124464089,ESSCIRC '88 Program Committee,10.1109/esscirc.1988.5468234,1988,ESSCIRC '88: Fourteenth European Solid-State C...,proceeding,1988-09,2020-02-02 00:24:01+00:00,NaN,0,None,None,None,None,closed,None
96,pub.1124464093,Fourteenth European Solid-State Circuits Confe...,10.1109/esscirc.1988.5468246,1988,ESSCIRC '88: Fourteenth European Solid-State C...,proceeding,1988-09,2020-02-02 00:24:01+00:00,NaN,0,None,None,None,None,closed,None
97,pub.1124464091,ESSDERC/ESSCIRC Organising Committee,10.1109/esscirc.1988.5468240,1988,ESSCIRC '88: Fourteenth European Solid-State C...,proceeding,1988-09,2020-02-02 00:24:01+00:00,NaN,0,None,None,None,None,closed,None
98,pub.1124464130,Integrated Circuits Digital Network (ISDN),10.1109/esscirc.1988.5468346,1988,ESSCIRC '88: Fourteenth European Solid-State C...,proceeding,1988-09,2020-02-02 00:24:01+00:00,NaN,0,None,None,None,None,closed,None
